# **calt Minimal Demo**

This notebook demonstrates the minimal code needed to:
1. Generate a dataset
2. Train a model
3. Evaluate the model

In [1]:
import sys
from pathlib import Path

# Add development calt to path (prioritize over pip-installed version)
# This notebook is in calt/examples/demos/, so we go up to calt/ and then to src/
# When running in Jupyter, Path.cwd() gives the notebook's directory
calt_dev_path = Path.cwd().parent.parent / "src"
sys.path.insert(0, str(calt_dev_path))

print(f"Using development calt from: {calt_dev_path}")

Using development calt from: /home/ara_shun/workspace/calt/src


## 1. Dataset Generation

<!-- Generate polynomial addition problems -->

In [ ]:
import random

from omegaconf import OmegaConf

from calt.dataset import DatasetPipeline
from calt.dataset.sympy.utils.polynomial_sampler import PolynomialSampler


# Define instance generator: polynomial addition
def polynomial_addition_generator(seed):
    random.seed(seed)

    # Initialize polynomial sampler
    sampler = PolynomialSampler(
        symbols="x0, x1",
        field_str="GF(7)",
        max_num_terms=2,
        max_degree=2,
        min_degree=1,
    )

    # Generate two polynomials
    F = sampler.sample(num_samples=2)

    # Solution is the sum
    g = sum(F)

    return F, g


# Load config from YAML file
cfg = OmegaConf.load("configs/data.yaml")

# Create dataset pipeline
pipeline = DatasetPipeline.from_config(
    cfg.dataset,
    instance_generator=polynomial_addition_generator,
)

# Run dataset generation
pipeline.run()

=========================== Dataset generation ===========================

Starting dataset generation for 2 dataset(s)
Dataset sizes: {'train': 10000, 'test': 1000}

---------------------------------- train ----------------------------------
Dataset size: 10000 samples  (Batch size: 10000)

--- Batch 1/1 ---
Processing samples 1-10000 (size: 10000)
Starting parallel processing...
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1249 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 2449 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 3199 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 4999 tasks      | elapsed:    1.1s

## 2. Model Training

<!-- Load data, create model, and train -->
<!-- The entire training pipeline can be summarized in just a few lines: -->


In [3]:
# Complete minimal training code
from omegaconf import OmegaConf

from calt.io import IOPipeline
from calt.models import ModelPipeline
from calt.trainer import TrainerPipeline

# Load config from YAML file
cfg = OmegaConf.load("./configs/train.yaml")

# Load data
io_pipeline = IOPipeline.from_config(cfg.data)
result = io_pipeline.build()

# Create model
model = ModelPipeline(cfg.model, result["tokenizer"]).build()

# Create trainer and train
trainer = TrainerPipeline(
    cfg.train,
    model=model,
    tokenizer=result["tokenizer"],
    train_dataset=result["train_dataset"],
    eval_dataset=result["test_dataset"],
    data_collator=result["data_collator"],
).build()

trainer.train()
success_rate = trainer.evaluate_and_save_generation()
print(f"Success rate: {100 * success_rate:.1f}%")

Loaded 10000 samples from ./data/train_raw.txt
Loaded 1000 samples from ./data/test_raw.txt
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/home/ara_shun/workspace/calt/src/calt/trainer/trainer.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Validating test dataset tokens... passed!


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/ara_shun/.netrc.
wandb: Currently logged in as: shun-arkw (chiba-u) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:268: DeprecationWarning: Read the `app_url` setting from the appropriate Settings object.
  app_url = wandb.util.app_url(tags["base_url"])  # type: ignore[index]
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:268: DeprecationWarning: Read the `app_url` setting from the appropriate Settings object.
  app_url = wandb.util.app_url(tags["base_url"])  # type: ignore[index]
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


{'loss': 3.4562, 'grad_norm': 2.586801052093506, 'learning_rate': 3.121019108280255e-05, 'epoch': 0.1597444089456869}
{'loss': 2.7497, 'grad_norm': 2.959867238998413, 'learning_rate': 6.305732484076433e-05, 'epoch': 0.3194888178913738}
{'loss': 1.9537, 'grad_norm': 3.6162636280059814, 'learning_rate': 9.490445859872612e-05, 'epoch': 0.4792332268370607}
{'loss': 1.4452, 'grad_norm': 2.590085506439209, 'learning_rate': 9.701704545454547e-05, 'epoch': 0.6389776357827476}
{'loss': 1.1572, 'grad_norm': 2.9573867321014404, 'learning_rate': 9.346590909090909e-05, 'epoch': 0.7987220447284346}
{'loss': 0.9915, 'grad_norm': 4.354540824890137, 'learning_rate': 8.991477272727273e-05, 'epoch': 0.9584664536741214}
{'loss': 0.8737, 'grad_norm': 4.0169525146484375, 'learning_rate': 8.636363636363637e-05, 'epoch': 1.1182108626198084}
{'loss': 0.7918, 'grad_norm': 3.4501237869262695, 'learning_rate': 8.28125e-05, 'epoch': 1.2779552715654952}
{'loss': 0.7427, 'grad_norm': 3.9847371578216553, 'learning_ra

Running evaluate_and_save_generation (step=1000, metric_key_prefix=eval)


{'eval_loss': 0.39848592877388, 'eval_token_accuracy': 0.8766434648105181, 'eval_success_rate': 0.221, 'eval_runtime': 0.1447, 'eval_samples_per_second': 6909.925, 'eval_steps_per_second': 221.118, 'epoch': 3.194888178913738}


Successfully saved generation results (step=1000, success_rate=0.2210)


{'eval_generation_success_rate': 0.221, 'eval_generation_step': 1000, 'epoch': 3.194888178913738}
{'loss': 0.468, 'grad_norm': 5.521849632263184, 'learning_rate': 3.6647727272727274e-05, 'epoch': 3.3546325878594248}
{'loss': 0.4518, 'grad_norm': 5.500566005706787, 'learning_rate': 3.3096590909090915e-05, 'epoch': 3.5143769968051117}
{'loss': 0.4576, 'grad_norm': 3.683093547821045, 'learning_rate': 2.954545454545455e-05, 'epoch': 3.6741214057507987}
{'loss': 0.4514, 'grad_norm': 4.174798488616943, 'learning_rate': 2.5994318181818182e-05, 'epoch': 3.8338658146964857}
{'loss': 0.4387, 'grad_norm': 4.234916687011719, 'learning_rate': 2.244318181818182e-05, 'epoch': 3.9936102236421727}
{'loss': 0.4402, 'grad_norm': 7.29291296005249, 'learning_rate': 1.8892045454545457e-05, 'epoch': 4.15335463258786}
{'loss': 0.4275, 'grad_norm': 4.153010845184326, 'learning_rate': 1.534090909090909e-05, 'epoch': 4.313099041533547}
{'loss': 0.4372, 'grad_norm': 4.2382049560546875, 'learning_rate': 1.17897727

## 3. Model Evaluation